In [ ]:
import pandas as pd

# Cargar CSV delimitado por |
df = pd.read_csv("MIGTABO19X.csv", delimiter="|", header=None)

# Renombrar columnas
df.columns = [
    "ro_operacion", "ro_concepto", "ro_signo", "ro_factor", "ro_referencial",
    "ro_signo_reajuste", "ro_factor_reajuste", "ro_referencial_reajuste",
    "ro_valor", "ro_porcentaje_efa", "ro_garantia", "ro_financiado"
]

# Normalizar conceptos
df["ro_concepto_normalizado"] = df["ro_concepto"].astype(str).str.strip().str.upper()

# Inicializar diccionario para errores
errores = {}

# Validación 1: ro_operacion
df["ro_operacion"] = pd.to_numeric(df["ro_operacion"], errors="coerce")
errores["ro_operacion"] = df[df["ro_operacion"].isnull() | (df["ro_operacion"] == 0)]

# Validación 2: ro_concepto
errores["ro_concepto"] = df[df["ro_concepto"].isnull() | (df["ro_concepto"].astype(str).str.strip() == "")]

# Validación 3: ro_signo
def validar_signo(row):
    signo = str(row["ro_signo"]).strip()
    concepto = row["ro_concepto_normalizado"]
    if concepto in ["INTERES", "INTERES MORA"]:
        return signo not in ["+", "-"]
    elif concepto == "CAPITAL":
        return pd.notna(row["ro_signo"]) and signo != ""
    return False

errores["ro_signo"] = df[df.apply(validar_signo, axis=1)]

# Validación 4: ro_factor (0 para CAPITAL)
errores["ro_factor"] = df[(df["ro_concepto_normalizado"] == "CAPITAL") & (df["ro_factor"] != 0)]

# Validación 5: ro_referencial (NULL para CAPITAL)
errores["ro_referencial"] = df[(df["ro_concepto_normalizado"] == "CAPITAL") & df["ro_referencial"].notna() & (df["ro_referencial"].astype(str).str.strip() != "")]

# Validación 6: ro_valor (>0 para CAPITAL, 0 para INTERES/INTERES MORA)
errores["ro_valor"] = df[
    ((df["ro_concepto_normalizado"] == "CAPITAL") & (df["ro_valor"] <= 0)) |
    ((df["ro_concepto_normalizado"].isin(["INTERES", "INTERES MORA"])) & (df["ro_valor"] != 0))
]

# Validación 7: ro_porcentaje_efa (>0 para INTERES/INTERES MORA, 0 para CAPITAL)
errores["ro_porcentaje_efa"] = df[
    ((df["ro_concepto_normalizado"] == "CAPITAL") & (df["ro_porcentaje_efa"] != 0)) |
    ((df["ro_concepto_normalizado"].isin(["INTERES", "INTERES MORA"])) & (df["ro_porcentaje_efa"] <= 0))
]

# Validación 8: ro_garantia = 0
errores["ro_garantia"] = df[df["ro_garantia"] != 0]

# Validación 9: ro_financiado = NULL
errores["ro_financiado"] = df[df["ro_financiado"].notna() & (df["ro_financiado"].astype(str).str.strip().str.upper() != "NULL")]

# Imprimir resultados
for campo, errores_df in errores.items():
    print(f"\nErrores en {campo} ({len(errores_df)} registros):")
    if errores_df.empty:
        print("✅ Sin errores")
    else:
        print(errores_df[["ro_operacion", "ro_concepto", campo]])
